In [87]:
%cd /home/parthgandhi/TradeBot

/home/parthgandhi/TradeBot


In [88]:
import polars as pl
import polars.selectors as cs
from src.config.storage_layout import StorageLayout
from src.config.market import Market
from src.config.exchange import Exchange
from src.config.data_source import DataSource
from src.config.brokers.nse import NSEConfig
from src.data_source.chartsmaze.helper import industry_to_sector
from src.data_source.chartsmaze.sectors import sectors as cmze_sectors

In [89]:
end_date = "2025-12-26"

# CMAZE File Fetch

In [59]:
cmaze_path = StorageLayout.data_dir(market=Market.INDIA, exchange=DataSource.CMAZE)
cmaze_sectors_df = industry_to_sector(mapping=cmze_sectors).lazy()

cmaze_df = (
    pl.scan_csv(cmaze_path / f"{end_date}.csv")
    .join(cmaze_sectors_df, on="Basic Industry", how="left")
    .group_by(pl.exclude("Sector"))
    .agg(pl.col("Sector").str.join(", "))
    .rename(
        {
            "Stock Name": "symbol",
            "RS Rating": "rs_rating",
            "Basic Industry": "basic_industry_cmaze",
            "Sector": "sector_cmaze",
            "Market Cap(Cr.)": "market_cap_cr_cmaze",
            "1 Month Returns(%)": "1_mo_rtr_pct",
            "3 Month Returns(%)": "3_mo_rtr_pct",
            "% from 52W High": "pct_from_52w_high",
        }
    ).select(['symbol',
 'rs_rating',
 'basic_industry_cmaze',
 'sector_cmaze',
 'market_cap_cr_cmaze',
 '1_mo_rtr_pct',
 '3_mo_rtr_pct',
 'pct_from_52w_high',
 ])
)

In [91]:
#  rclone sync /home/parthgandhi/TradeBot/storage/data/IND/ChartsMaze gdrive:Backup/SwingTrade/ChartsMaze/RS --progress

# NSE Sectors Fetch

In [63]:
db_path = StorageLayout.db_path(market=Market.INDIA, exchange=Exchange.NSE)

max_date_query = f"""
select max(timestamp) as timestamp
from '{NSEConfig.CLASSIFICATION_TABLE_ID}'
"""

max_date = pl.read_database_uri(query=max_date_query, uri=f"sqlite:///{db_path}").item(
    0, 0
)

industry_query = f"""
select *
from '{NSEConfig.CLASSIFICATION_TABLE_ID}'
where timestamp = '{max_date}'
"""

nse_classify_df = (
    pl.read_database_uri(query=industry_query, uri=f"sqlite:///{db_path}")
    .lazy()
    .rename({"timestamp": "latest_fetch_date"})
)

In [65]:
nse_classify_df.collect()

latest_fetch_date,symbol,macro_economic_sector,sector,industry,basic_industry,market_cap_cr
str,str,str,str,str,str,f64
"""2025-12-25""","""20MICRONS""","""Commodities""","""Metals & Mining""","""Minerals & Mining""","""Industrial Minerals""",695.71
"""2025-12-25""","""21STCENMGM""","""Financial Services""","""Financial Services""","""Capital Markets""","""Other Capital Market related S…",54.3
"""2025-12-25""","""360ONE""","""Financial Services""","""Financial Services""","""Capital Markets""","""Stockbroking & Allied""",48444.9
"""2025-12-25""","""3IINFOLTD""","""Information Technology""","""Information Technology""","""IT - Software""","""Computers - Software & Consult…",350.08
"""2025-12-25""","""3MINDIA""","""Diversified""","""Diversified""","""Diversified""","""Diversified""",38948.98
…,…,…,…,…,…,…
"""2025-12-25""","""RUDRA""","""Industrials""","""Capital Goods""","""Industrial Products""","""Iron & Steel Products""",256.78
"""2025-12-25""","""RUPA""","""Consumer Discretionary""","""Textiles""","""Textiles & Apparels""","""Garments & Apparels""",1284.72
"""2025-12-25""","""RUSHIL""","""Consumer Discretionary""","""Consumer Durables""","""Consumer Durables""","""Plywood Boards/ Laminates""",671.92


# Combine Filters

In [11]:
filters_path = StorageLayout.filters_dir(
    run_date=end_date, market=Market.INDIA_EQUITIES, exchange=Exchange.NSE
)

analysis_path = StorageLayout.analysis_dir(
    run_date=end_date, market=Market.INDIA_EQUITIES, exchange=Exchange.NSE
)

analysis_path.mkdir(parents=True, exist_ok=True)

basic_filter = (
    pl.scan_csv(filters_path / "basic_filter.csv")
    .with_columns(
        pl.col("timestamp")
        .str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S%.f")
        .cast(pl.Date)
        .alias("timestamp")
    )
    .select("timestamp", "symbol")
)


df_list = []
for filter_type in ["sma_200", "adr", "pullback", "reversal", "vcp"]:
    df = (
        pl.scan_csv(filters_path / f"{filter_type}_filter.csv")
        .with_columns(pl.lit(True).alias(f"{filter_type}_filter_flag"))
        .select("symbol", f"{filter_type}_filter_flag")
    )
    df_list.append(df)

# Combine Filters & CMAZE

In [66]:
res = (
    basic_filter.join(df_list[0], on="symbol", how="left")
    .join(df_list[1], on="symbol", how="left")
    .join(df_list[2], on="symbol", how="left")
    .join(df_list[3], on="symbol", how="left")
    .join(df_list[4], on="symbol", how="left")
    .with_columns(cs.ends_with("flag").fill_null(False))
    .join(cmaze_df, on="symbol", how="left")
    .join(nse_classify_df, on="symbol", how="left")
    .collect()
)

print(f"Before RS filter: {res.shape}")
res.write_csv(analysis_path / "overall_filter_result.csv")
res_gt_70 = res.filter(pl.col("rs_rating") >= 70)
print(f"After RS filter: {res_gt_70.shape}")

Before RS filter: (369, 20)
After RS filter: (254, 20)


In [67]:
res.head()

timestamp,symbol,sma_200_filter_flag,adr_filter_flag,pullback_filter_flag,reversal_filter_flag,vcp_filter_flag,rs_rating,basic_industry_cmaze,sector_cmaze,market_cap_cr_cmaze,1_mo_rtr_pct,3_mo_rtr_pct,pct_from_52w_high,latest_fetch_date,macro_economic_sector,sector,industry,basic_industry,market_cap_cr
date,str,bool,bool,bool,bool,bool,i64,str,str,i64,f64,f64,f64,str,str,str,str,str,f64
2025-12-26,"""21STCENMGM""",false,false,false,false,false,32,"""Investment Banking & Broking""","""Financial Services""",55,43.4,2.6,44.3,"""2025-12-25""","""Financial Services""","""Financial Services""","""Capital Markets""","""Other Capital Market related S…",54.3
2025-12-26,"""360ONE""",true,false,true,false,false,80,"""Asset Management""","""Financial Services""",47797,2.6,17.7,10.5,"""2025-12-25""","""Financial Services""","""Financial Services""","""Capital Markets""","""Stockbroking & Allied""",48444.9
2025-12-26,"""3MINDIA""",true,false,true,false,false,86,"""Diversified Operations""","""Diversified""",38966,-2.7,18.6,6.6,"""2025-12-25""","""Diversified""","""Diversified""","""Diversified""","""Diversified""",38948.98
2025-12-26,"""5PAISA""",false,true,false,false,false,38,"""Investment Banking & Broking""","""Financial Services""",1055,-1.7,-3.2,30.7,"""2025-12-25""","""Financial Services""","""Financial Services""","""Capital Markets""","""Stockbroking & Allied""",1087.08
2025-12-26,"""AARVI""",true,true,true,false,false,78,"""Diversified Commercial Service…","""Services""",190,-2.4,4.8,15.5,"""2025-12-25""","""Services""","""Services""","""Commercial Services & Supplies""","""Diversified Commercial Service…",192.39


In [85]:
industry_analysis = (
    res_gt_70.lazy()
    .group_by("sector_cmaze","basic_industry_cmaze")
    .agg(
        [pl.col("symbol").count().alias("symbols_count")]
        + [
            (
                (pl.col(f"{i}_mo_rtr_pct") * pl.col("market_cap_cr")).sum()
                / pl.col("market_cap_cr_cmaze").sum()
            )
            .round(2)
            .alias(f"{i}_mo_avg_weight_rtr_pct")
            for i in [1, 3]
        ]
        + [
            (
                (pl.col("rs_rating") * pl.col("market_cap_cr_cmaze")).sum()
                / pl.col("market_cap_cr_cmaze").sum()
            )
            .round(2)
            .alias(f"rs_rating_avg_weight")
        ]
    )
    .fill_nan(None)
    .with_columns(
        pl.mean_horizontal(cs.exclude("sector_cmaze", "basic_industry_cmaze", "symbols_count"))
        .round(2)
        .alias("industry_score"),
        (pl.col("1_mo_avg_weight_rtr_pct") * 100 / pl.col("3_mo_avg_weight_rtr_pct"))
        .round()
        .alias("1_by_3_rtr"),
    ).sort("symbols_count", descending=True)
    .collect()
)

In [86]:
industry_analysis

sector_cmaze,basic_industry_cmaze,symbols_count,1_mo_avg_weight_rtr_pct,3_mo_avg_weight_rtr_pct,rs_rating_avg_weight,industry_score,1_by_3_rtr
str,str,u32,f64,f64,f64,f64,f64
"""Auto""","""Auto Ancilaries""",18,0.02,1.53,86.67,29.41,1.0
"""IT""","""Software Services""",15,0.6,7.16,75.15,27.64,8.0
"""Financial Services""","""Private Banks""",14,2.02,12.84,83.52,32.79,16.0
"""Metals & Mining""","""Iron & Steel""",14,3.11,14.47,84.29,33.96,21.0
"""Capital Goods""","""Industrial Products & Manufact…",11,-0.83,9.05,88.77,32.33,-9.0
…,…,…,…,…,…,…,…
"""Telecommunication""","""Telecom - Infrastructure""",1,0.01,0.04,86.0,28.68,25.0
"""Healthcare""","""Biotechnology""",1,-0.81,16.81,83.0,33.0,-5.0
"""Healthcare, Chemicals""","""Medical Equipment & Supplies""",1,8.4,24.7,86.0,39.7,34.0


In [15]:
industry_analysis.write_csv(analysis_path / "industry_analysis.csv")

# RS >= 70 & Possible Upcoming Centers

In [16]:
industry_analysis.filter(
    (pl.col("rs_rating_avg_weight") >= 70)
    & (pl.col("1_mo_avg_weight_rtr_pct") >= pl.col("3_mo_avg_weight_rtr_pct"))
).sort("industry_score", descending=True)

basic_industry,symbols_count,1_mo_avg_weight_rtr_pct,3_mo_avg_weight_rtr_pct,rs_rating_avg_weight,industry_score,1_by_3_rtr
str,u32,f64,f64,f64,f64,f64
"""Media & Entertainment""",1,43.0,33.5,99.0,58.5,128.0
"""Packaging""",1,17.2,6.3,93.0,38.83,273.0
"""Railways""",1,19.9,14.8,73.0,35.9,134.0
"""Oil & Gas Drilling""",1,15.6,8.8,76.0,33.47,177.0


In [18]:
industry_analysis.filter((pl.col("rs_rating_avg_weight") >= 70)).sort(
    "industry_score", descending=True
).filter(pl.col("symbols_count") > 1)

basic_industry,symbols_count,1_mo_avg_weight_rtr_pct,3_mo_avg_weight_rtr_pct,rs_rating_avg_weight,industry_score,1_by_3_rtr
str,u32,f64,f64,f64,f64,f64
"""Mining/Minerals""",8,25.44,35.95,92.61,51.33,71.0
"""NBFC""",10,7.77,40.32,95.3,47.8,19.0
"""Tyres & Rubber Products""",3,8.16,35.67,93.29,45.71,23.0
"""Construction Products Miscalla…",3,9.42,32.32,88.84,43.53,29.0
"""Diversified Commercial Service…",3,10.17,30.77,85.78,42.24,33.0
…,…,…,…,…,…,…
"""Investment Banking & Broking""",5,3.82,9.87,77.13,30.27,39.0
"""Pharmaceuticals""",9,1.56,7.78,80.78,30.04,20.0
"""Tea & Coffee""",2,-7.4,5.99,88.77,29.12,-124.0
